企業の製品発売やプレスリリースの反応をチェックするのに，ツイッターなどの文章から感情分析(sentiment classification)を行いたい．  
これは意見マイニング(opinion mining)とも呼ばれ，研究開発が勧められている．  
前章のクラス分類の知識も応用して，自前で感情分類器を作る．  

# 本章のロードマップ
Twitterは文字数が短いため，特別な表記法や略語などが用いられる．  
パラグラフごとに感情に関する情報を集めて，その結果から文書全体の感情分析を行いたいが，できない．  
ここでは最先端の感情分類器までは踏み込まず，以下の3つのことに取り組む
- ナイーブベイズという分類手法を学ぶ
- 品詞タグ付け(Part-of-Speech tagging)について学ぶ
- scikit-learnのツールの使い方を学ぶ

# ツイートデータを取得する
ツイートとそれに対応するラベル(意見なし,ポジティブ，ネガティブ，無感情)一式が欲しい．  
Niek Sandersらによって5000以上のツイートに対して手作業でラベル付けが行われたコーパスを使用する．  
install.pyによってTwitterのサーバとやりとりしてデータをダウンロードできる．

まずはinstall.pyをコマンドラインで実行し，dataディレクトリにTweetとラベルのデータをcorpus.csvというファイル名で保存する．  
その後，util.pyにあるload_sanders_data()によってデータを読み込む

In [ ]:
import numpy as np
from ch06.utils import load_sanders_data()
X, Y = load_sanders_data()
classes = np.unique(Y) # 全ラベルの種類を取得
for c in classes:
    print("#%s: %i" % (c, sum(Y == c)))

以降では，意見なし(irrelevant)と感情なし(neutral)を同じものとして扱う

# ナイーブベイズ分類器の紹介
ナイーブベイズはもっとも洗練された機械学習アルゴリズムの一つと言える．  
出力と関係ない特徴量にロバスト，つまり無関係な特徴量をうまく無視する．  
学習，予測は高速で，メモリを必要としない．  
ナイーブと呼ばれるのは，全ての特徴量が互いに独立である，という過程が必要であるから．  
実際にはその過程が成立するのは稀であるがたとえそれが成立しなくても優れた性能を示すことが多い．  

マークダウンの左寄せが効かないのでHTMLで無理やり

In [8]:
%%html
<style>td, th{text-align: left !important}</style>

## ベイズ定理入門
ナイーブベイズ分類器が行なっているのは，クラスを分類するためにどの特徴量が証拠となるかということを追跡すること．  
ここでは，次の変数を例として使用する．  

|変数|取りうる値|意味|
| :- | :- | :- |
|C|"pos", "neg"|ツイートが属するクラス(ポジティブ，ネガティブ)|
|F1|Int(>=0)|ツイートで「awesome」という単語が用いられた回数|
|F2|Int(>=0)|ツイートで「crazy」という単語が用いられた回数|

ナイーブベイズでは，あるツイートについて，F1とF2がわかっている時そのツイートがクラスCに属する確率を求めている．  
この確率関数は$P(C | F_1, F_2)$と書くことができる．  
この確率は直接推定できないので，ベイズの定理を用いて式を変形する．  
ベイズの定理を以下に示す．  
$$ P(A) \cdot P(B|A) = P(B) \cdot P(A|B) $$
この式を使うと，  
$$ P(F_1, F_2) \cdot P(C | F_1, F_2) = P(C) \cdot P(F_1, F_2 | C) $$
これを変形し，左辺に$ P(C | F_1, F_2) $ がくるようにすると
$$ P(C | F_1, F_2) = \frac{P(C) \cdot P(F_1, F_2 | C)}{P(F_1, F_2)} $$
これは，次のように言い換えられる．  
- posterior: 事後確率, $P(C | F_1, F_2)$, $F_1$と$F_2$がわかっているとき，そのデータがクラス$C$に属する確率
- prior: 事前確率, $P(C)$, データについて何も情報がない場合にそのデータがクラス$C$に属する確率
- likelihood: 尤度, $P(F_1, F_2 | C)$, あるデータがクラス$C$に属することがわかっている場合，特徴量がその$F_1，F_2$である確率
- evidence: 証拠, $P(F_1, F_2)$, 特徴量がその$F_1$と$F_2$をとる確率, 該当する特徴量が全体に占める割合を計算して求める

という言葉を使って，
$$ posterior = \frac{prior \cdot likelihood}{evidence} $$

likelihoodを求めるのは少し考える必要がある．  

## ナイーブにする
確率理論($P(A, B) = P(B) \cdot P(A | B) $)から次の式が成り立つ
$$ P(F_1, F_2 | C) = P(F_1 | C) \cdot P(F_2 | C, F_1)$$
この式は，難しい問題$P(F_1, F_2 | C)$を別の難しい問題$P(F_2 | C, F_1)$に変換したにすぎない．  
<br>
ここで，$F_1$と$F_2$が独立であると仮定(ナイーブ)すると，$P(F_2 | C, F_1)$を$P(F_2 | C)$とかけるため，次のように書き直すことができる．  
$$ P(F_1, F_2 | C) = P(F_1 | C) \cdot P(F_2 | C)$$
<br>
これを合わせると，次の有用な式を得られる．  
$$ P(C | F_1, F_2) = \frac{P(C) \cdot P(F_1 | C) \cdot P(F_2 | C)}{P(F_1, F_2)} $$
<br>
独立の過程は都合的なもので理論的には正しくないが，現実のアプリケーションでは優れた結果になることが多くあるのが興味深い．

## ナイーブベイズを用いて分類を行う
新しいツイートが与えられた時に行うことは，単純に次の確率を計算することである．  
$$ P(C = "pos" | F_1, F_2) = \frac{P(C="pos") \cdot P(F_1 | C="pos") \cdot P(F_2 | C="pos")}{P(F_1, F_2)} $$
$$ P(C = "neg" | F_1, F_2) = \frac{P(C="neg") \cdot P(F_1 | C="neg") \cdot P(F_2 | C="neg")}{P(F_1, F_2)} $$
これらのうち，確率の大きい方を選ぶため，同じ式である分母は無視できる．  
この計算では，実際の確率には興味がなく，どちらのクラスの方があり得るか，ということに興味がある．  
このことが，ナイーブベイズのロバスト性を高めている理由になっている．  
<br>
以上を一般化すると，次の式によってツイートを分類できる．  
$$ C_{best} = \arg \max_{c \in C} P(C=c) \cdot P(F_1 | C=c) \cdot P(F_2 | C=c) $$

実際の例を使って計算を行なってみる．  
次のようなラベル付きデータが与えられているとする．  

|ツイート|クラス|
|:--|:--|
|awesome|pos|
|awesome|pos|
|awesome crazy|pos|
|crazy|pos|
|crazy|neg|
|crazy|neg|

このデータから，事前確率は，  
$$ P(C=pos) = \frac{4}{6} \approx 0.67 $$
$$ P(C=neg) = \frac{2}{6} \approx 0.33 $$
これは，あるツイートについて，その内容を何も知らない場合は，それはポジティブであると予測した方が良いということ．  
<br>
次に，各条件付き確率を求めていく．  
例えば，クラスがposであるとわかっている時，awesomeが存在する確率は次のように表せる．  
$$ P(F_1 > 0 | C=pos) = \frac{3}{4} = 0.75 $$
同様にして，
$$ P(F_2 > 0 | C=pos) = 0.5 $$
$$ P(F_1 > 0 | C=neg) = 0 $$
$$ P(F_2 > 0 | C=neg) = 1 $$
<br>
ここでは，試しにevidenceも計算し，実際の確率を求めてみる．  
$$ P(F_1, F_2) = P(F_1, F_2 | C=pos) \cdot P(C=pos) + P(F_1, F_2 | C=neg) \cdot P(C=neg) $$
実際には次の値になる．  
$$ P(F_1 > 1, F_2 > 1) = 0.22 $$
$$ P(F_1 > 1, F_2 > 0) = 0.44 $$
$$ P(F_1 > 0, F_2 > 1) = 0.33 $$
$$ P(F_1 > 0, F_2 > 0) = 0 $$
<br>
あとは，新しいツイートから特徴量$F_1$，$F_2$を抽出し，分類器に入力するだけ．  
awesome -> P: 0.57, N: 0 -> Positive  
crazy -> P: 0.25, N: 1 -> Negative  
awesome crazy -> P: 0.76, N: 0 -> Positive  
text -> P: ∞, N: ∞ （分母となるevidenceが0) -> 不確定  
<br>
この不確定な時にはどのように対応すれば良いか？  

## 新出単語への対応
上の例は明らかにデータが足りない．不確定な値も出てしまう．  
そこで，出現頻度に1を足すスムージング，(加算スムージング, additive smoothing, ラプラススムージング(Laplace Smoothing))を利用する．  
ポリゴンメッシュを滑らかにする画像処理のラプラシアンスムージングとは関係ない．  
調整可能な0以上のパラメータ$\alpha$をタスLidstoneスムージングという手法も存在する．  